In [ ]:
# < Genetic architecture of phenotypic differences between endangered hybridizing Arabis floodplain species >
# < PhD Thesis >
# < Neda Rahnamae> 

# < Chapter 2 - Genetic architecture of phenotypic differences between endangered hybridizing Arabis floodplain species >
# < Genotype Correction and Imputation >
# Feb 2024
# Window sizes should be adjusted

In [1]:
import pandas as pd
import numpy as np

In [2]:
def impute_zeros(array):
    # Copy of the array to not alter the original
    imputed_array = np.copy(array)
    for i in range(len(imputed_array)):
        if imputed_array[i] == 0:
            # Find the nearest non-zero values to the left
            left = i - 1
            while left >= 0 and imputed_array[left] == 0:
                left -= 1
            # Find the nearest non-zero values to the right
            right = i + 1
            while right < len(imputed_array) and imputed_array[right] == 0:
                right += 1
            # Determine which side to use for imputation based on availability
            if left >= 0 and right < len(imputed_array):
                # If both sides are available, use the nearest value
                imputed_array[i] = imputed_array[left] if (i - left) <= (right - i) else imputed_array[right]
            elif left >= 0:
                # If only left side is available
                imputed_array[i] = imputed_array[left]
            elif right < len(imputed_array):
                # If only right side is available
                imputed_array[i] = imputed_array[right]
    return imputed_array


def error_correction(positions_chr, plant_genotype_chr, window_size = 400_000, step_size = 100_000):
    
    mapping = {'-': 0, 'NN': 1, 'SS': 2, 'NS': 3}
    plant_genotype_codified_chr = [mapping[s] for s in plant_genotype_chr]
    positions_list = positions_chr
    encodings_list = plant_genotype_codified_chr
    positions_np = np.array(positions_list)
    
    def calculate_mode_skip_zero(values):
        # Exclude '0' from the values if it's the only or most frequent value
        values_filtered = [v for v in values if v != 0]
        if not values_filtered:  # If all values are '0', keep as is (no information to impute from)
            return 0
        counts = np.bincount(values_filtered)
        mode = np.argmax(counts)
        return mode
    
    # Re-implementing the sliding window logic with adjusted mode calculation
    imputed_values_np_skip_zero = np.full(positions_np.shape, -1)  # Using -1 as a placeholder again
    current_position = positions_np[0]
    max_position = positions_np[-1]
    while current_position < max_position:
        start_index = np.searchsorted(positions_np, current_position, side='left')
        end_position = current_position + window_size
        end_index = np.searchsorted(positions_np, end_position, side='right')
        window_encodings = [encodings_list[i] for i in range(start_index, end_index)]
        if window_encodings:
            mode_encoding = calculate_mode_skip_zero(window_encodings)
            imputed_values_np_skip_zero[start_index:end_index] = mode_encoding
        current_position += step_size
    return plant_genotype_codified_chr, imputed_values_np_skip_zero

In [ ]:
genotypes = pd.read_csv("map4_xxx_genotype.csv")

error_correction_plant_genotype_all_chr = []
unerror_correction_plant_genotype_all_chr = []

In [4]:
genotypes

,ID,chr,pos,1051,890,442,963,946,1117,475,...,627,717,587,742,156,139,815,271,775,327
0,chr1_1_0.020732,1,20732,-,NS,NS,-,NN,NS,NS,...,-,NN,NS,NN,NS,NS,NN,NN,NS,NN
1,chr1_1_0.022267,1,22267,NN,NS,NS,NN,NN,NS,NS,...,-,NN,NS,NN,-,NS,NS,-,NS,NN
2,chr1_1_0.054653,1,54653,NN,NS,NS,NN,NN,NS,NS,...,SS,-,NS,NN,-,NS,NS,NN,NS,NN
3,chr1_1_0.669375,1,669375,NN,NS,NS,-,NN,NS,NN,...,SS,NN,NS,-,SS,-,NS,NN,NS,-
4,chr1_1_0.669419,1,669419,NN,NS,NS,NN,NN,NS,NS,...,SS,-,NS,-,-,NS,NS,NN,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2725,chr8_8_37.247955,8,37247955,NN,NS,SS,SS,NN,SS,-,...,NS,NS,NS,NS,NN,SS,NS,NS,SS,-
2726,chr8_8_37.247975,8,37247975,NN,NS,SS,SS,NN,SS,-,...,NS,NS,NS,NS,NN,SS,NS,NS,SS,-
2727,chr8_8_38.085842,8,38085842,NN,NS,SS,SS,NS,NS,NS,...,NS,-,SS,-,NN,NN,NS,NS,SS,-
2728,chr8_8_38.705893,8,38705893,NN,NN,SS,SS,NS,NN,NN,...,-,NN,-,NS,NN,NN,NS,NN,SS,-


In [5]:
# Iterate over each column index
for col in range(3, 711):  # Assuming the relevant columns start from index 3
    genotypes_col_x = genotypes.iloc[:, col]  # Accessing column by index
    error_correction_plant_genotype_col_x = []
    unerror_correction_plant_genotype_col_x = []
    for chr in range(1, 9):
        genotypes_chr_x = genotypes[genotypes["chr"] == chr]
        positions_chr_x = genotypes_chr_x["pos"].tolist()
        plant_genotype_chr_x = genotypes_chr_x.iloc[:, col].tolist()  # Accessing column by index
        
        unerror_correction_plant_genotype_chr_x, error_correction_plant_genotype_chr_x = error_correction(positions_chr=positions_chr_x, plant_genotype_chr=plant_genotype_chr_x, window_size=400_000, step_size=100_000)
        error_correction_plant_genotype_col_x += error_correction_plant_genotype_chr_x.tolist()
        unerror_correction_plant_genotype_col_x += unerror_correction_plant_genotype_chr_x
    error_correction_plant_genotype_all_chr.append(error_correction_plant_genotype_col_x)
    unerror_correction_plant_genotype_all_chr.append(unerror_correction_plant_genotype_col_x)

error_correction_plant_genotype_all_chr = np.array(error_correction_plant_genotype_all_chr).T
unerror_correction_plant_genotype_all_chr = np.array(unerror_correction_plant_genotype_all_chr).T
imputed_error_correction_plant_genotype_all_chr = np.apply_along_axis(impute_zeros, 0, error_correction_plant_genotype_all_chr)

In [6]:
# Create a list to hold the columns
new_columns = []

# Iterate over each column index
for col in range(3, 711):  # Assuming the relevant columns start from index 3
    col_name = '{}_ei'.format(col)
    new_columns.append(pd.Series(imputed_error_correction_plant_genotype_all_chr[:, col-3], name=col_name))

# Concatenate all columns at once
genotypes = pd.concat([genotypes] + new_columns, axis=1)

In [7]:
# Save the corrected DataFrame to a new CSV file
genotypes.to_csv("corrected_genotypes_raw.csv", index=False)

In [8]:
# Selecting only the necessary columns
columns_to_keep = ['ID', 'chr', 'pos'] + [col_name for col_name in genotypes.columns[3:711]]
corrected_genotypes = genotypes[columns_to_keep].copy()

In [9]:
corrected_genotypes

,ID,chr,pos,1051,890,442,963,946,1117,475,...,627,717,587,742,156,139,815,271,775,327
0,chr1_1_0.020732,1,20732,-,NS,NS,-,NN,NS,NS,...,-,NN,NS,NN,NS,NS,NN,NN,NS,NN
1,chr1_1_0.022267,1,22267,NN,NS,NS,NN,NN,NS,NS,...,-,NN,NS,NN,-,NS,NS,-,NS,NN
2,chr1_1_0.054653,1,54653,NN,NS,NS,NN,NN,NS,NS,...,SS,-,NS,NN,-,NS,NS,NN,NS,NN
3,chr1_1_0.669375,1,669375,NN,NS,NS,-,NN,NS,NN,...,SS,NN,NS,-,SS,-,NS,NN,NS,-
4,chr1_1_0.669419,1,669419,NN,NS,NS,NN,NN,NS,NS,...,SS,-,NS,-,-,NS,NS,NN,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2725,chr8_8_37.247955,8,37247955,NN,NS,SS,SS,NN,SS,-,...,NS,NS,NS,NS,NN,SS,NS,NS,SS,-
2726,chr8_8_37.247975,8,37247975,NN,NS,SS,SS,NN,SS,-,...,NS,NS,NS,NS,NN,SS,NS,NS,SS,-
2727,chr8_8_38.085842,8,38085842,NN,NS,SS,SS,NS,NS,NS,...,NS,-,SS,-,NN,NN,NS,NS,SS,-
2728,chr8_8_38.705893,8,38705893,NN,NN,SS,SS,NS,NN,NN,...,-,NN,-,NS,NN,NN,NS,NN,SS,-


In [10]:
# Assigning corrected genotypes to the corresponding columns
for i, col_name in enumerate(corrected_genotypes.columns[3:]):
    corrected_genotypes[col_name] = imputed_error_correction_plant_genotype_all_chr[:, i]


In [11]:
# Save the DataFrame with only the necessary columns to a new CSV file
corrected_genotypes.to_csv("corrected_genotypes.csv", index=False)
